In [7]:
# Integración numérica de sistemas de ecuaciones diferenciales ordinarias
from scipy.integrate import solve_ivp

# Minimización de funciones objetivos para ajuste de parámetros
from scipy.optimize import least_squares
import numpy as np
import matplotlib.pyplot as plt
import copy

#### Definición sistema de ecuaciones diferenciales

In [8]:
# Función que es entregada a solve_ivp para resolver el modelo, se le entregara un BOOLEANO
# Para determinar si es carga o descarga
def adsorcion_hidrogeno(t, y, c_s, c_p, c_w, m_s, m_w, M_H2, R, alpha, beta, epsilon, volumen, area, p_0,n_0, b, m_dot, h, h_f, T_f, charge):
    
    if charge:
        print(f'Simulando proceso de carga\n--------------------------')
    
    # Desempacar variables:
    m_t = y[0]
    T = y[1]
    
    # Calcular Variables dependientes de las variables independientes
    # Moles de hidrógeno adsorbido
    n_a = n_0 * np.exp(-((R*T)/(alpha+beta*T))**b * np.log(p_0/p)**b)
    
    # Masa
    m_a = n_a*M_H2*m_s
    m_g = m_t - m_a
    
    # Presión
    p = (m_g*R*T)/(M_H2*volumen*epsilon)
    
    # Calor esostérico
    dH = alpha * (np.log(p_0/p))**(1/b)

    # ECUACIONES DIFERENCIALES
    # Ecuación diferencial de masa total
    if charge:
        dm_dt = m_dot
    elif not charge:
        dm_dt = -m_dot
        
    # Ecuación diferencial de temperatura
    if charge:
        dT_dt = (m_dot*h + dm_dt*(dH/M_H2) - h_f*area*(T-T_f))/(m_s*c_s + m_a*c_p + m_g*c_p + m_w*c_w)
    elif not charge:
        dT_dt = (-m_dot*h + dm_dt*(dH/M_H2) - h_f*area*(T-T_f))/(m_s*c_s + m_a*c_p + m_g*c_p + m_w*c_w)
    
    # Empacar el vector del lado derecho en un vector 2x1
    dy = np.array([dm_dt, dT_dt])
    
    return dy


#### Parametros, condiciones y solve_ivp

In [9]:
# PARAMETROS
# Calores especificos
c_s = 825 # Calor especifico del carbón activado (J kg-1 K-1)
c_p = 10167 # Calor especifico del hidrogeno (J kg-1 K-1)
c_w = 468 # Calor especifico paredes de acero (J kg-1 K-1)

# K: Originalmente estaba dado en J, por eso el /1000
# F: Sugerencia: ocupar unidades SI lo más posible

# Masas
m_s = 0.671 # masa carbón activado (kg)
m_w = 3.714 # Masa paredes de acero (kg)
M_H2 = 2.0159E-3 # Masa molar del hidregeno (kg mol-1)

# Constantes 
R = 8.314 # J mol-1 K-1
alpha = 3080 # Factor entalpico (J mol-1)
beta = 18.9 # Factor entropico (J mol-1 K-1)
epsilon_b = 0.49 
b = 2

# Dimensiones estanque
V = 2.4946*1e-3 # Volumen de estanque (m^3)
A_e = 0.1277 # Área superficial estanque m^2

# Otros
p_0 = 1470 * 1e6 # Presion de saturacion (Pa)
n_0 = 71.6 # Cantidad limite de adsorcion (mol kg-1)


# CONDICIONES DE BORDE
p_i = np.array([0.033, 0.033, 0.033, 0.032, 0.049, 0.032])*1e6 #Pa
T_i = np.array([281, 282, 280.2, 301.5, 302, 302.4]) #K 
T_f = np.array([282.5, 284.5, 282.2, 301.7, 302.5, 302.5]) #K
h_f = np.array([36, 36, 36, 36, 36, 36]) # W m^-2 K^-1

# Seleccionar los valores del test deseado
p_i = p_i[0]
T_i = T_i[0]
T_f = T_f[0]
h_f = h_f[0]

# Número de moles adsorbido en t = 0
n_a0 = n_0 * np.exp(- (R*T_i/(alpha + beta*T_i))**b * np.log(p_0/p_i)**b )
# n_a0 = 1e-6

print("---Condiciones iniciales---")
print("n_a0 = %.3f" % n_a0)
print(f'La presión incial es: {p_i}\nLa temperatura incial es: {T_i}\nLa temperatura final es: {T_f}\nh_f es: {h_f}')
print("---Fin CI---\n")


# PARAMETROS EN CARGA/DESCARGA
# Carga o descarga?
charge = True

# Por ahora solo se usara test No. 13
t_0 =  0            # s
t_f = 1042          # s
t_range = np.linspace(t_0, t_f, 1000000)

m_dot = 2.023e-5        # kg s-1
# Aparentemente calor de 
h = 3986.8          # J/kg aparentemente mal en el paper

# Tupla de parametros que se deben pasar a solve_ivp
args = (c_s, c_p, c_w, m_s, m_w, M_H2, R, alpha, beta, epsilon_b, V, A_e, p_0, n_0, b, m_dot, h, h_f, T_f, charge)

# En ingeniería química, sobre todo cuando hay reacciones o cambio de fases, se generan sistemas ultraestables
# (ultrastiff) no conviene utilizar métodos explícitos. 
sol = solve_ivp(adsorcion_hidrogeno, (t_0, t_f), [n_a0, T_i, p_i], args=args, t_eval= t_range, method = 'BDF')

---Condiciones iniciales---
n_a0 = 0.010
La presión incial es: 33000.0
La temperatura incial es: 281.0
La temperatura final es: 282.5
h_f es: 36
---Fin CI---

Simulando proceso de carga
--------------------------


UnboundLocalError: local variable 'p' referenced before assignment